In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
from matplotlib import pyplot as plt
import os

一个用于生成某一层节点的函数

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
def build_single_blocl(filter_arr,kernel_size_arr,stride_arr,inputs,times):

    x=inputs;
    for i in range(0,times):
        x=tf.keras.layers.Conv2D(filters=filter_arr[0],kernel_size=(kernel_size_arr[0],kernel_size_arr[0]),strides=(stride_arr[0],stride_arr[0]),kernel_initializer='he_normal',padding='same')(x)
        x=tf.keras.layers.BatchNormalization(axis=3)(x)
        x=tf.keras.layers.Activation('elu')(x)

        x=tf.keras.layers.Conv2D(filters=filter_arr[1],kernel_size=(kernel_size_arr[1],kernel_size_arr[1]),strides=(stride_arr[1],stride_arr[1]),kernel_initializer='he_normal',padding='same')(x)
        x=tf.keras.layers.BatchNormalization(axis=3)(x)
        x=tf.keras.layers.Activation('elu')(x)
    
        output=tf.keras.layers.add([inputs,x])
        output=tf.nn.relu(output)

    return output

def build_tripple_block(filter_arr,kernel_size_arr,stride_arr,inputs,times):
    x=inputs;
    for i in range(0,times):
        x=tf.keras.layers.Conv2D(filters=filter_arr[0],kernel_size=(kernel_size_arr[0],kernel_size_arr[0]),strides=(stride_arr[0],stride_arr[0]),kernel_initializer='he_normal',padding='same')(x)
        x=tf.keras.layers.BatchNormalization(axis=3)(x)
        x=tf.keras.layers.Activation('elu')(x)

        x=tf.keras.layers.Conv2D(filters=filter_arr[0],kernel_size=(kernel_size_arr[1],kernel_size_arr[1]),strides=(stride_arr[1],stride_arr[1]),kernel_initializer='he_normal',padding='same')(x)
        x=tf.keras.layers.BatchNormalization(axis=3)(x)
        x=tf.keras.layers.Activation('elu')(x)

        x=tf.keras.layers.Conv2D(filters=filter_arr[1],kernel_size=(kernel_size_arr[2],kernel_size_arr[2]),strides=(stride_arr[2],stride_arr[2]),kernel_initializer='he_normal',padding='same')(x)
        x=tf.keras.layers.BatchNormalization(axis=3)(x)
        x=tf.keras.layers.Activation('elu')(x)
    
        output=tf.keras.layers.add([inputs,x])
        output=tf.nn.relu(output)

    return output

In [ ]:
os.chdir("/gdrive/MyDrive/ISBI_Dataset")
X_ids = next(os.walk('train'))[2]
Y_ids = next(os.walk('label'))[2]
print(len(X_ids),len(Y_ids))
X_ids.sort()
Y_ids.sort()

In [ ]:
IMG_CHANNELS, IMG_WIDTH, IMG_HEIGHT = 3, 512, 512

In [ ]:
def vertical_symmetry(img_input):
    return np.array(img_input)[::-1,:,:]
    
def horizental_symmetry(img_input):
    return np.array(img_input)[:,::-1,:]

def vertical_symmetry_2D(img_input):
    return np.array(img_input)[::-1,:]
    
def horizental_symmetry_2D(img_input):
    return np.array(img_input)[:,::-1]
    
def cropping(img_input):
    return img_input.crop((0,0,img_input.width/2,img_input.height/2)),img_input.crop((0,img_input.height/2-1,img_input.width/2,img_input.height-1)),img_input.crop((img_input.width/2-1,0,img_input.width-1,img_input.height/2-1)),img_input.crop((img_input.width/2-1,img_input.height/2-1,img_input.width-1,img_input.height-1))

In [ ]:
X_train = np.zeros((len(X_ids)*7, 256, 256, 3), dtype=np.float32)
Y_train = np.zeros((len(Y_ids)*7, 256, 256, 1), dtype=np.bool)
X_original=np.zeros((len(X_ids),512,512,3),dtype=np.float32)
Y_original=np.zeros((len(Y_ids),512,512,1),dtype=np.float32)

n=0
m=0
for id_ in (X_ids):
    image = tf.keras.preprocessing.image.load_img(f'/gdrive/MyDrive/ISBI_Dataset/train/{id_}', target_size=(IMG_HEIGHT, IMG_WIDTH))
    # print(n,id_)
    input_arr = tf.keras.preprocessing.image.img_to_array(image)[90:450,150:406]
    image = tf.keras.preprocessing.image.array_to_img(input_arr, ).resize((256, 256))
    original_image=tf.keras.preprocessing.image.array_to_img(input_arr, ).resize((512, 512))
    X_train[n] = np.array(image)
    X_original[m]=np.array(original_image)
    img1,img2,img3,img4=cropping(tf.keras.preprocessing.image.array_to_img(X_original[m]))
    n+=1
    X_train[n]=np.array(img1.resize((256, 256)))
    n+=1
    X_train[n]=np.array(img2.resize((256, 256)))
    n+=1
    X_train[n]=np.array(img3.resize((256, 256)))
    n+=1
    X_train[n]=np.array(img4.resize((256, 256)))
    n+=1
    X_train[n]=np.array(tf.keras.preprocessing.image.array_to_img(vertical_symmetry(tf.keras.preprocessing.image.array_to_img(X_train[])),).resize((256, 256)))
    n+=1
    X_train[n]=np.array(tf.keras.preprocessing.image.array_to_img(horizental_symmetry(tf.keras.preprocessing.image.array_to_img(X_train[temp])),).resize((256, 256)))
    n+=1
    m+=1

n=0
m=0
for  id_ in (Y_ids):
    image = tf.keras.preprocessing.image.load_img(f'/gdrive/MyDrive/ISBI_Dataset/label/{id_}', 
                                                  target_size=(IMG_HEIGHT, IMG_WIDTH), color_mode="grayscale")

    print(n,id_)
    input_arr = tf.keras.preprocessing.image.img_to_array(image)[90:450,150:406]
    image = tf.keras.preprocessing.image.array_to_img(input_arr, ).resize((256, 256))
    image = tf.keras.preprocessing.image.array_to_img(input_arr, ).resize((256, 256))
    original_image=tf.keras.preprocessing.image.array_to_img(input_arr, ).resize((512, 512))
    Y_train[n] = np.array(image)[:, :, np.newaxis]
    Y_original[m]=np.array(original_image)[:, :, np.newaxis]
    
    img1,img2,img3,img4=cropping(tf.keras.preprocessing.image.array_to_img(Y_original[m]))
    n+=1
    Y_train[n]=np.array(img1.resize((256, 256)))[:, :, np.newaxis]
    n+=1
    Y_train[n]=np.array(img2.resize((256, 256)))[:, :, np.newaxis]
    n+=1
    Y_train[n]=np.array(img3.resize((256, 256)))[:, :, np.newaxis]
    n+=1
    Y_train[n]=np.array(img4.resize((256, 256)))[:, :, np.newaxis]
    n+=1
    Y_train[n]=np.array(vertical_symmetry_2D(tf.keras.preprocessing.image.array_to_img(Y_train[temp])),)[:, :, np.newaxis]
    n+=1
    Y_train[n]=np.array(horizental_symmetry_2D(tf.keras.preprocessing.image.array_to_img(Y_train[temp])),)[:, :, np.newaxis]
    n+=1
    m+=1

In [ ]:
ax1 = plt.subplot(121)
ax1.imshow(tf.keras.preprocessing.image.array_to_img(X_train[29]))
ax2=plt.subplot(122)
ax2.imshow(tf.keras.preprocessing.image.array_to_img(Y_train[29]))
plt.show()

In [ ]:
inputs=tf.keras.layers.Input((512,512,3))
c0=tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='elu',kernel_initializer='he_normal',padding='same')(inputs)
p0=tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=(2,2))(c0)
Deconv2=tf.keras.layers.Conv2DTranspose(filters=64,kernel_size=(4,4),strides=(2,2),kernel_initializer='he_normal')(p0)

block1=build_single_block(filter_arr=[128,128],kernel_size_arr=[3,3],stride_arr=[2,1],inputs=p0,times=1)
block1=build_single_block(filter_arr=[128,128],kernel_size_arr=[3,3],stride_arr=[1,1],inputs=block1,times=2)
Deconv3=tf.keras.layers.Conv2DTranspose(filters=64,kernel_size=(8,8),strides=(4,4),kernel_initializer='he_normal',padding='same')(block1)

block2=build_single_block(filter_arr=[256,256],kernel_size_arr=[3,3],stride_arr=[2,1],inputs=block1,times=1)
block2=build_single_block(filter_arr=[256,256],kernel_size_arr=[3,3],stride_arr=[1,1],inputs=block2,times=1)

block3=build_single_block(filter_arr=[512,512],kernel_size_arr=[3,3],stride_arr=[1,1],inputs=block2,times=6)

block4=build_single_block(filter_arr=[512,1024],kernel_size_arr=[3,3],stride_arr=[1,1],inputs=block3,times=3)

# block5=build_single_block(filter_arr=[512,1024],kernel_size_arr=[1,3],stride_arr=[1,1],inputs=block4,times=1)
# block6=build_single_block(filter_arr=[2048,1024],kernel_size_arr=[1,1],stride_arr=[1,1],inputs=block5,times=1)
# block6=build_single_block(filter_arr=[2048,4096],kernel_size_arr=[3,1],stride_arr=[1,1],inputs=block6,times=1)

block5=build_tripple_block(filter_arr=[512,1024,2048],kernel_size_arr=[1,3,1],stride_arr=[1,1,1],inputs=block4,times=1)
block6=build_tripple_block(filter_arr=[1024,2048,4096],kernel_size_arr=[1,3,1],stride_arr=[1,1,1],inputs=block5,times=1)


Deconv1=tf.keras.layers.Conv2DTranspose(filters=64,kernel_size=(16,16),strides=(8,8),padding='same',kernel_initializer='he_normal')(block6)

fusion=tf.add(Deconv1,Deconv2)
fusion=tf.add(fusion,Deconv3)

outConv1=tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),kernel_initializer='he_normal',padding='same')(fusion)
d1=tf.keras.layers.Dropout(rate=0.5)(outConv1)
outConv2=tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),strides=(1,1),kernel_initializer='he_normal',padding='same')(d1)
outputs=tf.keras.layers.Softmax()(outConv2)

model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.summary()

In [ ]:
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=8, epochs=1000)
model.save("CASIA_MIRA.h5")

In [ ]:
plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import random

test_id = random.choice(X_ids)
print(test_id)
img = tf.keras.preprocessing.image.load_img(f"/gdrive/MyDrive/ISBI_Dataset/train/{test_id}", target_size=(256, 256))
input_array = tf.keras.preprocessing.image.img_to_array(img)
input_array_model = np.array([input_array])
predictions = model.predict(input_array_model)
ax10=plt.subplot(121)
ax10.imshow(np.asarray(Image.open(f"/gdrive/MyDrive/ISBI_Dataset/train/{test_id}")))
ax11=plt.subplot(122)
ax11.imshow(tf.keras.preprocessing.image.array_to_img(np.squeeze(predictions)[:, :, np.newaxis]))
plt.show()

In [ ]:
ax12=plt.subplot(121)
ax12.imshow(np.asarray(tf.keras.preprocessing.image.array_to_img(np.squeeze(predictions)[:, :, np.newaxis])))
ax13=plt.subplot(122)
train_id=test_id.replace('volume','labels')
ax13.imshow(np.asarray(Image.open(f"/gdrive/MyDrive/ISBI_Dataset/label/{train_id}")))
plt.show()